In [ ]:
from final_exploration import *
from final_modeling import *


alpha=0.05
target='logerror'


random.seed(123)


In [ ]:



partitionslist=partitionslist_with_scaled(scaled_vars = ['latitude', 'longitude', 'bathroomcnt', 'taxrate'])


In [ ]:

#Explore 

train_la=partitionslist[0][0]
orange_train=partitionslist[1][0]
ventura_train=partitionslist[2][0]


Question 1:

Is there a correlation between square footage of a home and log error?



In [ ]:
pearsonsRsquareLogError(train_la,'structure_dollar_sqft_bin')


Question 2:

Is there a relationship between tax rate and log error?


In [ ]:
pearsonsRsquareLogError(train_la,'taxrate')


Question 3:

Does log error vary by when the house was sold?

In [ ]:
interestingcols=['age']
samepopdict,diffpopdict,lengthsame,lengthdiff=granulartwocombocomparison(train_la,interestingcols,target,n=100)



ageandlogrelpie(lengthsame,lengthdiff)

Since age is actually a categorical, we explored every unique age with a sample size of greater than 100.

(a conservative sample size to ensure our tests are significant).

 We ran the levene test then a ttest for indpendence respective to the levene test results. The net results give us a bit more detail than an ANOVA which would allow for deeper inspection.
 
 Considering an ANOVA only would answer if any of the samples differ, we can see that it is the case that our logerror overwhelmingly does not vary with age (in LA county).

Question 4,5,6:


Is Log error is significantly different among the counties of LA County, Orange County and Ventura County?

We actually ask three seperate questions here as we compare LA to OC, LA to Ventura and Ventura to OC.


In [ ]:
threeQandA_stats_viz_counties(train_la,orange_train,ventura_train)
    

We see that LA and OC come from statistically different populations. However Ventura is not indepedent of LA or OC. 

It should be noted that the sizes of these subpopulations differ by a large degree. As we represent with the pie chart below.

In [ ]:
county_train_pie(train_la,orange_train,ventura_train)

In [ ]:
X=partitionslist[0][1:4]

train=partitionslist[0][0]

In [ ]:






# list of variables I will cluster on. 
cluster_vars = ['scaled_latitude', 'scaled_longitude', 'age_bin']
cluster_name = 'area_cluster'
k_range = range(2,20)





In [ ]:


find_k(X[0], cluster_vars, k_range)



k = 5
# as reminders: 
cluster_vars = ['scaled_latitude', 'scaled_longitude', 'age_bin']
cluster_name = 'area_cluster'

kmeans = create_clusters(X[0], k, cluster_vars)





It appears that k such that k equals 6,7 or 8 is my best  fit.

I am not sure how to obtain that k....

In [ ]:

centroid_df = get_centroids(kmeans, cluster_vars, cluster_name)
centroid_df


In [ ]:




X = assign_clusters(kmeans, cluster_vars, cluster_name, centroid_df,X)


In [ ]:

pd.DataFrame(X[0].groupby(['area_cluster', 'centroid_scaled_latitude', 'centroid_scaled_longitude', 
                           'centroid_age_bin'])['area_cluster'].count())




In [ ]:

cluster_vars = ['scaled_bathroomcnt', 'sqft_bin', 'acres_bin', 'bath_bed_ratio']
cluster_name = 'size_cluster'
k_range = range(2,20)



In [ ]:
X[0][cluster_vars].isna().sum()

In [ ]:


k=7
cluster_name = 'size_cluster'
cluster_vars = ['scaled_bathroomcnt', 'sqft_bin', 'acres_bin', 'bath_bed_ratio']

# fit kmeans 
kmeans = create_clusters(X[0], k, cluster_vars)

# get centroid values per variable per cluster
centroid_df = get_centroids(kmeans, cluster_vars, cluster_name)

# get cluster assignments and append those with centroids for each X partition (train, validate, test)
X = assign_clusters(kmeans, cluster_vars, cluster_name, centroid_df,X)







X_train = X[0].iloc[:,0:27]

xtrainlist=X_train.columns.to_list()
# xtrainlist


In [ ]:


pd.DataFrame(X_train[X_train.size_cluster==6].groupby(['size_cluster', 'centroid_scaled_bathroomcnt', 'centroid_sqft_bin',
                              'centroid_acres_bin', 'centroid_bath_bed_ratio'])['size_cluster'].count())



# Laz 
### This is all me trying to figue out how to fit the clusters. 
### Still no idea. Once we get the cluster we need, look like 6 we can fit it to the model. Do you know?

In [ ]:
# grp1=X_train.groupby(['size_cluster', 'centroid_scaled_bathroomcnt', 'centroid_sqft_bin',
#                               'centroid_acres_bin', 'centroid_bath_bed_ratio'])['size_cluster']



In [ ]:

# list(grp1.groups.keys())[-1]

In [ ]:
# pd.DataFrame(grp1.groups.get((6, 0.31645721125074855, 0.5468222621184924, 0.2206822262118493, 0.708588526972728)))



In [ ]:

# plt.figure(figsize=(12,6))
# plt.scatter(y=X_train.latitude, x=X_train.longitude, c=X_train.area_cluster, alpha=.4)
# plt.scatter(y=X_train.age, x=X_train.longitude, c=X_train.area_cluster, alpha=.4)



In [ ]:

y_train = partitionslist[0][-3]


# y_train

In [ ]:


# plt.scatter(y=y_train.logerror, x=X_train.age, c=X_train.area_cluster, alpha=.7)
# plt.ylim(-1,1)
# plt.xlabel('Age of Property')
# plt.ylabel('Log Error of Zestimate')
# plt.title("Do clusters reveal differences in age and error?")



In [ ]:



# sns.boxplot(y=y_train.logerror, x=X_train.area_cluster)
# plt.ylim(-1, 1)
# # sns.swarmplot(X_train.age_bin, y_train.logerror, hue=X_train.area_cluster)

# plt.figure(figsize=(12,6))
# # plt.scatter(y=X_train.latitude, x=X_train.longitude, c=X_train.area_cluster, alpha=.4)
# plt.scatter(y=y_train.logerror, x=X_train.calculatedfinishedsquarefeet, c=X_train.size_cluster, alpha=.7)
# plt.yscale('symlog')
# plt.xlabel('Finished Square Feet')
# plt.ylabel('Log Error of Zestimate')
# plt.title('Is there distinction between clusters when visualizing size of the home by the error in zestimate?')

# plt.show()



In [ ]:


plt_df = X_train[['bathroomcnt', 'calculatedfinishedsquarefeet', 
                    'acres', 'bath_bed_ratio', 'size_cluster']]

# sns.pairplot(data=plt_df, hue='size_cluster')



# sns.scatterplot(x='bath_bed_ratio', y='calculatedfinishedsquarefeet', 
#                 data=X_train, hue='size_cluster')

# update datatypes of binned values to be float
X_train = X_train.astype({'size_cluster': 'category', 'area_cluster': 'category'})


dummy_df = pd.get_dummies(X_train[['size_cluster','area_cluster']], dummy_na=False, drop_first=[True, True])

# append dummy df cols to the original df. 
X_train = pd.concat([X_train, dummy_df], axis=1)


# plt.scatter(X_train.longitude, X_train.latitude, c = X_train.area_cluster)



In [ ]:

# X_train.groupby(['area_cluster_id', 'size_cluster_id'])['structure_dollar_per_sqft']
X_train.head()

In [ ]:







cols=['scaled_latitude', 'scaled_longitude', 'scaled_bathroomcnt', 'scaled_taxrate']
dbscancols=[ 'scaled_longitude', 'scaled_bathroomcnt',  'dbscan']
kmeanscols=[ 'scaled_longitude', 'scaled_bathroomcnt',  'kmeans']



partitionslist=partitionslist_with_scaled(scaled_vars = ['latitude', 'longitude', 'bathroomcnt', 'taxrate'])
mvp=['scaled_latitude', 'scaled_longitude', 'scaled_bathroomcnt',
       'scaled_taxrate']

train,X_train, X_validate, X_test, y_train, y_validate, y_test=partitionslist[0]

#This step is to ensure we only send scaled data to the model
MVPlist=mvpXforModels(partitionslist,mvp)
X_train, X_validate, X_test=MVPlist[0]       

rmseDF=regmodelbest(X_train, X_validate, X_test, y_train, y_validate, y_test,random=123)
rmseDF
LArmseDF=LATest(X_train,X_validate,X_test,y_train,y_validate,y_test)

LArmseDF



In [ ]:


dbscan = DBSCAN(eps=0.123, min_samples = 2)
X_train['dbscan'] = dbscan.fit_predict(X_train[cols])
X_validate['dbscan'] = dbscan.fit_predict(X_validate[cols])
X_test['dbscan'] = dbscan.fit_predict(X_test[cols])
# kmeans_degree6 = pf6.transform(kmeans_degree6)



kmeans = KMeans(n_clusters=6)
X_train['kmeans'] = kmeans.fit_predict(X_train[cols])
X_validate['kmeans'] = kmeans.fit_predict(X_validate[cols])
X_test['kmeans'] = kmeans.fit_predict(X_test[cols])
# dbscan_degree6 = pf6.transform(dbscan_degree6)



In [ ]:

dbclusterRSME=LATest(X_train[dbscancols],X_validate[dbscancols],X_test[dbscancols],y_train,y_validate,y_test)

dbclusterRSME




In [ ]:


KmeansclusterRSME=LATest(X_train[kmeanscols],X_validate[kmeanscols],X_test[kmeanscols],y_train,y_validate,y_test)

KmeansclusterRSME
